# Introduction to OpenSearch

OpenSearch is search engine that nicely scales to billion size documents. Its indexes can be composed of multiple fields, each one indexing different parts of the documents. Each field has its own data type, e.g., text, keywords, numbers, knn-vectors. Text fields use a specific analyser and a retrieval model.

A server is available on the cluster for this course. You can also set up your own server in your local machine. Docker is a convenient solution: https://opensearch.org/docs/latest/opensearch/install/docker/

## OpenSearch connection settings

For this course, a server is available on the cluster for this course. If you really need to you can set up your own server in your local machine. I advise you to use docker: https://opensearch.org/docs/latest/opensearch/install/docker/


In [44]:
import pprint as pp
import requests
import os

host = 'api.novasearch.org'
port = 443
user = 'user212'
password = 'soO-2518'
index_name = user

## OpenSearch Python API 

OpenSearch communicates via REST which can be accessed with CURL (or its Python port, the requests library). For your conveninence we will use the Python client. A short introduction is available here:
https://opensearch.org/docs/latest/clients/python/

## Opening and closing a connection
The example below establishes a connection with the server and, if your index is already created, it displays the index settings, mappings and the number of indexed documents.

In [45]:
import pprint as pp
from opensearchpy import OpenSearch
from opensearchpy import helpers

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

if client.indices.exists(index_name):

    resp = client.indices.open(index = index_name)
    print(resp)

    print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
    settings = client.indices.get_settings(index = index_name)
    pp.pprint(settings)

    print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
    mappings = client.indices.get_mapping(index = index_name)
    pp.pprint(mappings)

    print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
    print(client.count(index = index_name))
    

To release resources in the OpenSearch server you should always close the index handle.

In [3]:
resp = client.indices.close(index = index_name, timeout=600)
print(resp)

{'acknowledged': True, 'shards_acknowledged': True, 'indices': {'user212': {'closed': True}}}


# Index creation and configuration
In this section we will see how to create an index, inspect the configuration and delete an index if needed.

## Create an index with your own settings

Lets first create an index distributed across 4 shards, no replicas, and with support for knn-vector data types. In terms of indexed data, we define two data properties: doc_id and contents, with data types keyword and text respectively.

Property type | Description
-----|-----
text|A string sequence of characters that represent full-text values.
keyword|A string sequence of structured characters, such as an email or ZIP code.
boolean|OpenSearch accepts true and false as boolean values. An empty string is equal to false. 
integer|A signed 32-bit number. 
float|A single-precision 32-bit floating point number. 
double|A double-precision 64-bit floating point number.
date|if new string fields match a date’s format, then the string is processed as a date field. For example, date: "2012/03/11" is processed as a date.
objects|Objects are standard JSON objects, which can have fields and mappings of their own. For example, a movies object can have additional properties such as title, year, and director.


In [46]:

index_body = {
   "settings":{
      "index":{
         "number_of_replicas":0,
         "number_of_shards":4,
         "refresh_interval":"-1",
         "knn":"true"
      }
   },
   "mappings":{
       "dynamic":      "strict",
       "properties":{
         "doc_id":{
            "type":"keyword"
         },
         "tags":{
            "type":"keyword"
         },
         "title":{
            "type":"text",
            "analyzer":"standard",
            "similarity":"BM25"
         },
         "title_embedding":{
            "type":"knn_vector",
            "dimension": 768,
            "method": {
                "name": "hnsw",
                "space_type": "innerproduct",
                "engine": "faiss",
                "parameters": {
                    "ef_construction": 256,
                    "m": 48
                }
            }
         },
         "description":{
            "type":"text",
            "analyzer":"standard",
            "similarity":"BM25"
         },
         "description_embedding":{
            "type":"knn_vector",
            "dimension": 768,
            "method": {
                "name": "hnsw",
                "space_type": "innerproduct",
                "engine": "faiss",
                "parameters": {
                    "ef_construction": 256,
                    "m": 48
                }
            }
         },
         "ingredients":{
           "type":"text",
            "analyzer":"standard",
            "similarity":"BM25"
         },
         "time":{
           "type":"integer"  
         },
         "difficultyLevel":{
            "type": "keyword"
         },
         "nutrients": {
            "type":"text",
            "analyzer":"standard",
            "similarity":"BM25"
         },
         "contents":{
            "type":"text",
            "analyzer":"standard",
            "similarity":"BM25"
         }
      }
   }
}

if client.indices.exists(index=index_name):
    print("Index already existed. Nothing to be done.")
else:        
    response = client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)



Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'user212'}


## Check the indexes, settings and mappings
Once you create an index, you should verify that it is created according to your requirements.

In [47]:
print('\n----------------------------------------------------------------------------------- INDEX SETTINGS')
index_settings = {
    "settings":{
      "index":{
         "refresh_interval" : "1s"
      }
   }
}
client.indices.put_settings(index = index_name, body = index_settings)
settings = client.indices.get_settings(index = index_name)
pp.pprint(settings)

print('\n----------------------------------------------------------------------------------- INDEX MAPPINGS')
mappings = client.indices.get_mapping(index = index_name)
pp.pprint(mappings)

print('\n----------------------------------------------------------------------------------- INDEX #DOCs')
print(client.count(index = index_name))



----------------------------------------------------------------------------------- INDEX SETTINGS
{'user212': {'settings': {'index': {'creation_date': '1712516198711',
                                    'knn': 'true',
                                    'number_of_replicas': '0',
                                    'number_of_shards': '4',
                                    'provided_name': 'user212',
                                    'refresh_interval': '1s',
                                    'uuid': 'Kheod5SjS6OUNYxiWWNm2g',
                                    'version': {'created': '135238227'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'user212': {'mappings': {'dynamic': 'strict',
                          'properties': {'contents': {'analyzer': 'standard',
                                                      'similarity': 'BM25',
                                                      'type': 'text'},
           

## Index deletion
Be absolutely sure tha you want to delete the index. There is no way of recovering it!

In [5]:
be absolutely sure that you want to comment this line and actually delete the index!!!

if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = 600
    )
    print('\nDeleting index:')
    print(response)


Deleting index:
{'acknowledged': True}


# Document processing and indexing

## Built-in document tokenizers and analyzers

The built-in tokenizers and analyzers include: standard, simple, whitespace, stop, keyword, pattern, [language], fingerprint.


In [15]:
anls = {
  "analyzer": "whitespace",
  "text": "the quick brown fox"
}
client.indices.analyze(body=anls, index=index_name)

{'tokens': [{'token': 'the',
   'start_offset': 0,
   'end_offset': 3,
   'type': 'word',
   'position': 0},
  {'token': 'quick',
   'start_offset': 4,
   'end_offset': 9,
   'type': 'word',
   'position': 1},
  {'token': 'brown',
   'start_offset': 10,
   'end_offset': 15,
   'type': 'word',
   'position': 2},
  {'token': 'fox',
   'start_offset': 16,
   'end_offset': 19,
   'type': 'word',
   'position': 3}]}

In [39]:
anls = {
  "analyzer": "standard",
  "text": "the quick brown fox"
}
client.indices.analyze(body=anls, index=index_name)


{'tokens': [{'token': 'the',
   'start_offset': 0,
   'end_offset': 3,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'quick',
   'start_offset': 4,
   'end_offset': 9,
   'type': '<ALPHANUM>',
   'position': 1},
  {'token': 'brown',
   'start_offset': 10,
   'end_offset': 15,
   'type': '<ALPHANUM>',
   'position': 2},
  {'token': 'fox',
   'start_offset': 16,
   'end_offset': 19,
   'type': '<ALPHANUM>',
   'position': 3}]}


## Simple document indexing

You can index documents in OpenSearch by adding one document at a time. This is done with JSON data as follows. Note that the id parameter in the API call is the unique identifier, which works as a key.

If you add one document that already exists in the index, it will update the data. You can update only part of the fields and leave all the others unchanged.


In [48]:
import json

titles = []
descriptions = []
with open('receitas.json', encoding='utf-8') as f:
    data = json.loads(f.read())

for doc in data:
    doc_idx = int(doc)
    doc_info = data[doc]

    ingredients = ""
    for ingredient in doc_info['ingredients']:
        if (ingredient["ingredient"]):
            ingredients += ingredient["ingredient"] + ' '  

    if (doc_info['nutrition']):
        if (doc_info['nutrition']['nutrients']):
            nutrients = "" + str(doc_info['nutrition']['nutrients'])

    titles.append(doc_info['displayName'])
    if (doc_info['description'] != None):
        descriptions.append(doc_info['description'])
    
    obj = {
        'doc_id': doc_idx,
        'tags': ['recipe'],
        'title': doc_info['displayName'],
        'description': doc_info['description'],
        'ingredients': ingredients,
        'nutrients': nutrients,
        'contents': str(doc_info)
    }
    # if time is filled add to obj
    if (doc_info['totalTimeMinutes']): obj['time'] = doc_info['totalTimeMinutes']
    # if difficultyLevel is filled add to obj
    if (doc_info['difficultyLevel']): obj['difficultyLevel'] = doc_info['difficultyLevel']
    
    resp = client.index(index=index_name, id=doc_idx, body=obj)
    print(resp['result'])



created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created


## Deleting a single document
Similarly, you can delete one document at a time as shown below.

In [ ]:
Delete the document.

response = client.delete(
    index = index_name,
    id = id
)

print('\nDeleting document:')
print(response)


# Search methods

OpenSearch supports many different search methods and has its own Query Syntax Language covering a wide range of search options:

https://opensearch.org/docs/latest/opensearch/query-dsl/index/

## Text-based search

OpenSearch is one of the best solutions for searching text. The text-based search documentation is available here:

https://opensearch.org/docs/latest/opensearch/query-dsl/full-text/

In the example below the 'query'  parameter indicates the search query, the 'size' parameter indicates the number of documents to be returned, the parametner 'source' indicates which fields should be returned in the search results, and the 'fields' parameter indicates the list of fields to be searched. 


In [53]:
qtxt = "chicken"

query_bm25 = {
  'size': 5,
  '_source': ['doc_id'],
#  '_source': ['doc_id'],
#  '_source': '',
  'query': {
    'multi_match': {
      'query': qtxt,
      'fields': ['title', 'description']
    }
  }
}

response = client.search(
    body = query_bm25,
    index = index_name
)

print('\nSearch results:')
pp.pprint(response)



Search results:
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': '0',
                    '_index': 'user212',
                    '_score': 2.5257938,
                    '_source': {'doc_id': 0},
                    '_type': '_doc'}],
          'max_score': 2.5257938,
          'total': {'relation': 'eq', 'value': 1}},
 'timed_out': False,
 'took': 4}



## Term queries

https://opensearch.org/docs/latest/opensearch/query-dsl/term/


In [59]:
qtxt = "How many people live in London?"

query_bm25 = {
  'size': 5,
  '_source': ['contents'],
  'query': {
        "term": {
            "tags" : 'red'
        }
   }
}

response = client.search(
    body = query_bm25,
    index = index_name
)

print('\nSearch results:')
pp.pprint(response)



Search results:
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [],
          'max_score': None,
          'total': {'relation': 'eq', 'value': 0}},
 'timed_out': False,
 'took': 2}



## Boolean queries

https://opensearch.org/docs/latest/opensearch/query-dsl/bool/


In [38]:
qtxt = "How many people live in London?"

query_bm25 = {
  'size': 5,
  '_source': ['contents'],
#  '_source': ['doc_id'],
#  '_source': '',
  'query': {
      'bool':{
          "must":{
            "term": {
                "tags" : 'red'
            }
          },
          "should": 
        {
            'multi_match': {
              'query': qtxt,
              'fields': ['contents']
            }
        }
      }
  }
}

response = client.search(
    body = query_bm25,
    index = index_name
)

print('\nSearch results:')
pp.pprint(response)



Search results:
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [],
          'max_score': None,
          'total': {'relation': 'eq', 'value': 0}},
 'timed_out': False,
 'took': 2}


In [50]:
# compute embedding vectors
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

# encode recipe vectors
titles_emb = encode(titles)
description_emb = encode(descriptions)    

In [51]:
# place vector indexes
for idx in range(0, len(titles_emb)):
    doc = {
        'doc_id': 'document-' + str(idx),
        'contents': titles[idx],
        'title_embedding': titles_emb[idx].numpy()
    }
    resp = client.index(index=index_name, id=idx, body=doc)
    print(resp['result'])

# place vector indexes
for idx in range(0, len(description_emb)):
    doc = {
        'doc_id': 'document-' + str(idx),
        'contents': titles[idx],
        'description_embedding': description_emb[idx].numpy()
    }
    resp = client.index(index=index_name, id=idx, body=doc)
    print(resp['result'])

updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated


In [52]:
# Compute the query embedding
query = "I want a chicken based recipe"
query_emb = encode(query)

query_denc = {
  'size': 5,
#  '_source': ['doc_id', 'contents', 'sentence_embedding'],
  '_source': ['doc_id', 'contents'],
#  '_source': ['doc_id'],
   "query": {
        "knn": {
          "title_embedding": {
            "vector": query_emb[0].numpy(),
            "k": 2
          }
        }
      }
}

response = client.search(
    body = query_denc,
    index = index_name
)

print('\nSearch results:')
pp.pprint(response)


Search results:
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': '743',
                    '_index': 'user212',
                    '_score': 1.5065033,
                    '_source': {'contents': 'Skillet Chicken with Potatoes and '
                                            'Mushrooms',
                                'doc_id': 'document-742'},
                    '_type': '_doc'},
                   {'_id': '800',
                    '_index': 'user212',
                    '_score': 1.5049436,
                    '_source': {'contents': 'One Whole Chicken, Three '
                                            'Different Meals',
                                'doc_id': 'document-799'},
                    '_type': '_doc'},
                   {'_id': '127',
                    '_index': 'user212',
                    '_score': 1.4927741,
                    '_source': {'contents': 'How to Cook: Whole Grilled '
                        

# Approximate KNN search in semantic embeddings

## Create an index with dense vectors

To create the dense vector field you can use the configuration provided below. The 'dimension' property indicates the dimensionality of the indexed vectors, the 'space_type' indicates similarity function, and the parameters are specific to the indexing method, the 'hnsw' (Hierarichical Navigable Small World).

For details see the https://opensearch.org/docs/latest/search-plugins/knn/approximate-knn/

In [ ]:
index_body = {
   "settings":{
      "index":{
         "number_of_replicas":0,
         "number_of_shards":4,
         "refresh_interval":"-1",
         "knn":"true"
      }
   },    
   "mappings":{
      "dynamic":      "strict",
      "properties":{
         "doc_id":{
            "type":"keyword"
         },
         "contents":{
            "type":"text",
            "analyzer": "standard",
#            "analyzer":"my_analyzer",
            "similarity":"BM25"
         },
         "sentence_embedding":{
            "type":"knn_vector",
            "dimension": 768,
            "method":{
               "name":"hnsw",
               "space_type":"innerproduct",
               "engine":"faiss",
               "parameters":{
                  "ef_construction":256,
                  "m":48
               }
            }
         }
      }
   }
}

if client.indices.exists(index=index_name):
    print("Index already existed. You may force the new mappings.")
else:        
    response = client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)


## Dual-Encoders

To compute the embedding vectors of each document, we can use the Transformer encodres trained in the MSMARCO Dataset. There are many other models available in the HuggingFace repository.

In [15]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

# Sentences we want sentence embeddings for
docs = ["Around 9 Million people live in London", "London is known for its financial district"]
doc_emb = encode(docs)


## Indexing document embedding vectors

In the previous step we saw how to compute the embedding representation of a document. You can index document embeddings in OpenSearch by adding a new field to your JSON file as shown below.


In [16]:
doc = {
    'doc_id': 'documentA',
    'contents': docs[0],
    'sentence_embedding': doc_emb[0].numpy()
}
resp = client.index(index=index_name, id=1, body=doc)
print(resp['result'])

doc = {
    'doc_id': 'documentB',
    'contents': docs[1],
    'sentence_embedding': doc_emb[1].numpy()
}
resp = client.index(index=index_name, id=2, body=doc)
print(resp['result'])


created
created


## Embedding spaces search

Similarly, you need to compute the embedding representation of the query and submit it in the search query as shown next.

In [18]:
# Compute the query embedding
query = "How many people live in London?"
query_emb = encode(query)

query_denc = {
  'size': 5,
#  '_source': ['doc_id', 'contents', 'sentence_embedding'],
#  '_source': ['doc_id', 'contents'],
  '_source': ['doc_id'],
   "query": {
        "knn": {
          "sentence_embedding": {
            "vector": query_emb[0].numpy(),
            "k": 2
          }
        }
      }
}

response = client.search(
    body = query_denc,
    index = index_name
)

print('\nSearch results:')
pp.pprint(response)



Search results:
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'user219',
                    '_score': 1.9538686,
                    '_source': {'doc_id': 'documentA'},
                    '_type': '_doc'},
                   {'_id': '2',
                    '_index': 'user219',
                    '_score': 1.434623,
                    '_source': {'doc_id': 'documentB'},
                    '_type': '_doc'}],
          'max_score': 1.9538686,
          'total': {'relation': 'eq', 'value': 2}},
 'timed_out': False,
 'took': 27}


## Training dual-encoders

You can fine-tune dual-encoders in your domain data and get some extra points out of your search architecture.

      https://www.sbert.net/docs/training/overview.html
    

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

#Define your train examples. You need more than just two examples...
train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
    InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

# Specialized document analyzers
Text data can be represented in different ways. In this section we will see different ways of transforming natural language documents into a computational representations.

## Create an index with specific analyzers
The built-in analyzers offer a wide range of text processing methods. Each field can have its own analyzer and users can also define their own analyzers.

In [7]:

index_body = {
   "settings":{
      "index":{
         "number_of_replicas":0,
         "number_of_shards":4,
         "refresh_interval":"-1",
         "knn":"true"
      },
      "analysis":{
         "filter":{
            "edge_ngram_filter":{
               "type":"edge_ngram",
               "min_gram":1,
               "max_gram":20
            }
         },
         "analyzer":{
            "my_analyzer":{
               "type":"custom",
               "tokenizer":"standard",
               "filter":[
                  "lowercase",
                  "edge_ngram_filter"
               ]
            }
         }
      }
   },
   "mappings":{
      "properties":{
         "doc_id":{
            "type":"keyword"
         },
         "contents":{
            "type":"text",
            "analyzer":"my_analyzer",
            "similarity":"BM25"
         }
      }
   }
}

if client.indices.exists(index=index_name):
    print("Index already existed. Nothing to be done.")
else:        
    response = client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)



Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'user220'}


## Spacy: Tokens, lemmas and POS
If the built-in text processing methods are not sufficient for your problem, you can use external libraries like Spacy to extract other representations of text, such as POS and lemmas, or to use other stemmers.

In [13]:
import spacy
from spacy import displacy
from pathlib import Path

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

save_figures = False

print("token".ljust(10), "lemma".ljust(10), "pos".ljust(6), "tag".ljust(6), "dep".ljust(10),
            "shape".ljust(10), "alpha", "stop")
print("------------------------------------------------------------------------------")
for token in doc:
    print(token.text.ljust(10), token.lemma_.ljust(10), token.pos_.ljust(6), token.tag_.ljust(6), token.dep_.ljust(10),
            token.shape_.ljust(10), token.is_alpha, token.is_stop)


token      lemma      pos    tag    dep        shape      alpha stop
------------------------------------------------------------------------------
Apple      Apple      PROPN  NNP    nsubj      Xxxxx      True False
is         be         AUX    VBZ    aux        xx         True True
looking    look       VERB   VBG    ROOT       xxxx       True False
at         at         ADP    IN     prep       xx         True True
buying     buy        VERB   VBG    pcomp      xxxx       True False
U.K.       U.K.       PROPN  NNP    compound   X.X.       False False
startup    startup    NOUN   NN     dobj       xxxx       True False
for        for        ADP    IN     prep       xxx        True True
$          $          SYM    $      quantmod   $          False False
1          1          NUM    CD     compound   d          False False
billion    billion    NUM    CD     pobj       xxxx       True False


## Named entity recognition
Spacy also identifies the mentions of relevant named entities. This can have a number of applications, such as selecting the documents that mention a given named entity.

In [14]:
import spacy
from spacy import displacy
from pathlib import Path

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text.ljust(12), ent.label_.ljust(10), ent.start_char, ent.end_char)

html_ent = displacy.render(doc, style="ent", jupyter=True)


Apple        ORG        0 5
U.K.         GPE        27 31
$1 billion   MONEY      44 54
